## Deliverable 2. Create a Customer Travel Destinations Map.

In [29]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

In [30]:
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [31]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Patricio,US,28.0170,-97.5169,68.41,51,0,8.63,clear sky
1,1,Aklavik,CA,68.2191,-135.0107,1.40,78,75,5.75,light snow
2,2,Ushuaia,AR,-54.8000,-68.3000,47.86,66,75,25.32,broken clouds
3,3,Esperance,AU,-33.8667,121.9000,69.64,60,59,8.57,light rain
4,4,Port Hedland,AU,-20.3167,118.5667,82.45,83,40,17.27,scattered clouds


In [32]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [33]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Port Hedland,AU,-20.3167,118.5667,82.45,83,40,17.27,scattered clouds
8,8,Touros,BR,-5.1989,-35.4608,88.70,63,22,15.86,few clouds
10,10,Inhambane,MZ,-23.8650,35.3833,78.78,78,20,4.61,few clouds
12,12,Bubaque,GW,11.2833,-15.8333,78.78,56,23,12.64,few clouds
13,13,Puerto Ayora,EC,-0.7393,-90.3518,82.36,71,100,9.46,overcast clouds
21,21,Saint-Philippe,RE,-21.3585,55.7679,80.19,77,68,11.54,broken clouds
22,22,Curvelo,BR,-18.7564,-44.4308,85.80,48,99,10.18,overcast clouds
24,24,Butaritari,KI,3.0707,172.7902,81.70,78,74,21.27,broken clouds
26,26,Bambous Virieux,MU,-20.3428,57.7575,80.85,78,75,14.97,light rain
28,28,Atuona,PF,-9.8000,-139.0333,76.87,79,9,12.10,light rain


In [34]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                190
City                   190
Country                189
Lat                    190
Lng                    190
Max Temp               190
Humidity               190
Cloudiness             190
Wind Speed             190
Current Description    190
dtype: int64

In [35]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [36]:
clean_preferred_cities_df = preferred_cities_df.dropna()

In [37]:
clean_preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Port Hedland,AU,82.45,scattered clouds,-20.3167,118.5667,
8,Touros,BR,88.70,few clouds,-5.1989,-35.4608,
10,Inhambane,MZ,78.78,few clouds,-23.8650,35.3833,
12,Bubaque,GW,78.78,few clouds,11.2833,-15.8333,
13,Puerto Ayora,EC,82.36,overcast clouds,-0.7393,-90.3518,
21,Saint-Philippe,RE,80.19,broken clouds,-21.3585,55.7679,
22,Curvelo,BR,85.80,overcast clouds,-18.7564,-44.4308,
24,Butaritari,KI,81.70,broken clouds,3.0707,172.7902,
26,Bambous Virieux,MU,80.85,light rain,-20.3428,57.7575,
28,Atuona,PF,76.87,light rain,-9.8000,-139.0333,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [40]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Port Hedland,AU,82.45,scattered clouds,-20.3167,118.5667,The Esplanade Hotel (Accommodation section clo...
8,Touros,BR,88.70,few clouds,-5.1989,-35.4608,INN NEW HORIZON
10,Inhambane,MZ,78.78,few clouds,-23.8650,35.3833,Oceano
12,Bubaque,GW,78.78,few clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
13,Puerto Ayora,EC,82.36,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
...,...,...,...,...,...,...,...
682,Quatre Cocos,MU,80.71,broken clouds,-20.2078,57.7625,Sunrise Attitude Hotel
684,Nauta,PE,89.31,overcast clouds,-4.5061,-73.5757,Amazon Exploring Expedition - Eretzen Tá Lodge
685,Port Keats,AU,80.85,broken clouds,-14.2500,129.5500,TDC Village (Hundred Man Camp)
691,Ekpoma,NG,81.93,overcast clouds,6.7421,6.1390,Alice Hotels


In [41]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   189
Country                189
Max Temp               189
Current Description    189
Lat                    189
Lng                    189
Hotel Name             189
dtype: int64

In [42]:
hotel_df[hotel_df['Hotel Name'] == '']

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
55,Tari,NG,86.18,broken clouds,12.4086,4.4914,
124,Adrar,MR,82.49,overcast clouds,20.5022,-10.0711,
149,Birao,CF,75.02,scattered clouds,10.2849,22.7882,
199,Kidal,ML,77.13,clear sky,18.4411,1.4078,
201,Taoudenni,ML,79.90,overcast clouds,22.6783,-3.9836,
223,Ler,SS,81.37,broken clouds,8.3018,30.1418,
267,Cuautepec,MX,85.19,clear sky,16.7000,-99.0000,
297,Madimba,TZ,76.86,broken clouds,-10.4000,40.3333,
331,Bere,TD,78.71,scattered clouds,9.3202,16.1552,
347,Merauke,ID,77.68,overcast clouds,-8.4667,140.3333,


In [43]:
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']

In [44]:
clean_hotel_df.count()

City                   170
Country                170
Max Temp               170
Current Description    170
Lat                    170
Lng                    170
Hotel Name             170
dtype: int64

In [45]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Port Hedland,AU,82.45,scattered clouds,-20.3167,118.5667,The Esplanade Hotel (Accommodation section clo...
8,Touros,BR,88.70,few clouds,-5.1989,-35.4608,INN NEW HORIZON
10,Inhambane,MZ,78.78,few clouds,-23.8650,35.3833,Oceano
12,Bubaque,GW,78.78,few clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
13,Puerto Ayora,EC,82.36,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
...,...,...,...,...,...,...,...
681,San Policarpo,PH,77.77,few clouds,12.1791,125.5072,Agapita M. Nicart
682,Quatre Cocos,MU,80.71,broken clouds,-20.2078,57.7625,Sunrise Attitude Hotel
684,Nauta,PE,89.31,overcast clouds,-4.5061,-73.5757,Amazon Exploring Expedition - Eretzen Tá Lodge
685,Port Keats,AU,80.85,broken clouds,-14.2500,129.5500,TDC Village (Hundred Man Camp)


In [50]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [47]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(20.0, 21.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))